<h1 style="text-align: center;">Homework 4</h1>

In [2]:
import numpy as np 
import pandas as pd

import requests
from bs4 import BeautifulSoup
from pprint import pprint
import time

import json
from lxml import html
from lxml.etree import tostring
from xml.dom.minidom import parseString
from dicttoxml import dicttoxml

In [3]:
base_url = "https://www.youtube.com"
url = "https://www.youtube.com/feed/trending"

In [4]:
response = requests.get(url) #headers={"user-agent": "edu_research_bot"}
page = response.content
tree = html.document_fromstring(page)

In [8]:
[i.text_content().replace(" views", '') for i in tree.xpath("//ul[@class='yt-lockup-meta-info']/li")[1]]

[]

In [21]:
tree.xpath("//ul[@class='yt-lockup-meta-info']/li[2]")[0].text_content()

'1,317,831 views'

In [37]:
trending = {}
divs = tree.xpath("//div[@class='yt-lockup-content']/h3/a/@href")
trending["URL"] = tree.xpath("//a[@class='yt-uix-tile-link yt-ui-ellipsis yt-ui-ellipsis-2 yt-uix-sessionlink      spf-link ']/@href")
trending["Title"] = tree.xpath("//a[@class='yt-uix-tile-link yt-ui-ellipsis yt-ui-ellipsis-2 yt-uix-sessionlink      spf-link ']/@title")
trending["Duration"] = [i.text_content().replace("- Duration: ", '') for i in tree.xpath("//span[@class='accessible-description']")]
trending["Username"] = [i.text_content() for i in tree.xpath("//a[@class='yt-uix-sessionlink       spf-link ']")]
trending["Views"] = [i.text_content().replace(" views", '') for i in tree.xpath("//ul[@class='yt-lockup-meta-info']/li[2]")]

In [38]:
[key +":" + str(len(trending[key])) for key in trending.keys()] 
#YouTube added some 'Creator on the rise' video table in trending list, that's why the lengths are different

['URL:69', 'Title:69', 'Duration:74', 'Username:74', 'Views:74']

In [39]:
trending["Username"][4:9]

['Trent Palmer',
 'Trent Palmer',
 'Trent Palmer',
 'Trent Palmer',
 'Trent Palmer']

In [40]:
del(trending["Duration"][4:9])
del(trending["Username"][4:9])
del(trending["Views"][4:9])
trending["URL"] = [base_url + i for i in trending["URL"]]

In [41]:
trending_df = pd.DataFrame(trending)
trending_df.Duration = trending_df.Duration.str.replace(".", '')
trending_df.Views = trending_df.Views.str.replace(",", '')
trending_df.Views = trending_df.Views.apply(pd.to_numeric)
trending_df.head()

,URL,Title,Duration,Username,Views
0,https://www.youtube.com/watch?v=MBPdKxlazD0,Blind Chicken Wing Taste Test,14:29,Good Mythical Morning,1317831
1,https://www.youtube.com/watch?v=w7pYhpJaJW8,Alita: Battle Angel | Official Trailer [HD] | ...,2:37,20th Century Fox,2590262
2,https://www.youtube.com/watch?v=f3uVr2bZKUU,Sara Haines Announces She's Leaving 'The View'...,4:55,The View,375595
3,https://www.youtube.com/watch?v=tgBHu2D66Rg,5 things you need to know before tonight's Raw...,1:31,WWE,258924
4,https://www.youtube.com/watch?v=PV9jdolesMI,Game Theory: We Were Right ALL ALONG! (FNAF Ul...,18:32,The Game Theorists,3682972


In [43]:
print("Trending mean:", np.mean(trending_df.Views))

Trending mean: 3504611.434782609
